# 古典线性回归

## 导入样本数据

In [1]:
import pandas as pd

df = pd.read_excel('./数据/上证指数与沪深300.xlsx')
df.head()

,日期,hs300,sz
0,2019-11-22,3849.9948,2885.2884
1,2019-11-21,3889.5980,2903.6379
2,2019-11-20,3907.8641,2911.0534
3,2019-11-19,3947.0392,2933.9908
4,2019-11-18,3907.9291,2909.2002


## 原理讲解

### 古典线性回归模型的假定

#### 假定1：线性假定

总体模型为:$y_i=\beta_1x_{i1}+\beta_2x_{i2}+\cdots+\beta_Kx_{iK}+\varepsilon_i (i=1,\cdots,n)\tag{1}$            

其中 $n$ 为样本容量，解释变量 $x_{ik}$ 的第一个下标表示第 $i$ 个“观测值”，而第二个下标则表示第 $k$ 个解释变量$(k=1,\cdots,K)$，共有 $K$个 解释变量。如果有常数项，则通常令第一个解释变量为单位向量，即 $x_{i1}=1$

为了更简洁地表达，下面引入矩阵符号。把方程（1）的所有解释变量和参数都写成向量，记第 $i$ 个观测数据为$x_i\equiv\left(x_{i 1} ,x_{i_{2}} \cdots x_{i K}\right)^{\prime}$，$\beta \equiv\left(\beta_{1}, \beta_{2} \cdots \beta_{K}\right)^{\prime}$，则方程（1）为：

$y_{i}=x_{i}^{\prime} \boldsymbol{\beta}+\varepsilon_{i} \quad(i=1, \cdots, n)\tag{2}$

#### 假定2：严格外生性

$\mathrm{E}\left(\varepsilon_{i} | X\right)=\mathrm{E}\left(\boldsymbol{\varepsilon}_{i} | x_{1}, \cdots, x_{n}\right)=0 \quad(i=1, \cdots, n)\tag{3}$

#### 假定3：不存在“严格多重共线性”（strict multicolinearity），即数据矩阵 $X$ 满列秩

#### 假定4：球型扰动项（spherical disturbance），即扰动项满足“同方差”、“无自相关”的性质

$\operatorname{Var}(\boldsymbol{\varepsilon} | X)=\mathrm{E}\left(\boldsymbol{\varepsilon} \boldsymbol{\varepsilon}^{\prime} | X\right)=\sigma^{2} \boldsymbol{I}_{n}=\left(\begin{array}{ccc}\sigma^{2} & & 0 \\ & \ddots & \\ 0 & & \sigma^{2}\end{array}\right)\tag{4}$

### 最小二乘法
假定待估计方程为：$hs300 = c+sz$。其中 c 为常数项

#### OLS估计量b

$$b \equiv\left(X^{\prime} X\right)^{-1} X^{\prime} y\tag{5}$$

In [2]:
import numpy as np

n = df.shape[0]   # 样本容量
beta = np.array(df['sz']).reshape(n,1)
c = np.ones((n,1))   # 常数项
X = np.hstack((c,beta))   # hstack()在行上合并，vstack()在列上合并
y = np.array(df['hs300']).reshape(n,1)

b = np.linalg.inv(X.T @ X) @ X.T @ y
print('OLS估计值为：\n',b)

OLS估计值为：
 [[-124.69031687]
 [   1.29305435]]


#### 残差 $e$

$$e \equiv\left(\begin{array}{llll}
e_{1},e_{2},\cdots,e_{n}
\end{array}\right)=y-X \widetilde{\beta}\tag{6}$$

In [3]:
e = y - X @ b

#### 扰动项方差 $s^{2}$

对于扰动项方差$\sigma^{2}=\operatorname{Var}\left(\varepsilon_{i}\right)$，由于总体扰动项 $\varepsilon$ 不可观测，而样本残差 $e$ 可以近似地看成是 $\varepsilon$ 的实现值，故使用以下统计量作为对方差 $\sigma^{2}$ 的估计：
$$s^{2} \equiv \frac{1}{n-K} \sum_{i=1}^{n} e_{i}^{2}\tag{7}$$

In [4]:
K = X.ndim
SSE = e.T @ e
s2 = SSE/(n-K) 

import math 
s = math.sqrt(s2)

print('平方和：', SSE)
print('扰动项方差', s2)
print('扰动项标准差', s)

平方和： [[5453855.41555515]]
扰动项方差 [[11907.98125667]]
扰动项标准差 109.12369704454953


#### 估计量 b 的方差-协方差矩阵

$$\operatorname{Var}(b | X)=\boldsymbol{\sigma}^{2}\left(X^{\prime} X\right)^{-1}\tag{8}$$

In [5]:
Varb = s2 * np.linalg.inv(X.T @ X)

print('协方差矩阵：\n', Varb)

协方差矩阵：
 [[ 3.77128774e+03 -1.27819004e+00]
 [-1.27819004e+00  4.36206925e-04]]


#### 置信区间

由于 $\frac{b_{k}-\beta_{k}}{\mathrm{SE}\left(b_{k}\right)} \sim t(n-K)$，根据 $t_{\alpha/2}$ 得：
$$\mathrm{P}\left\{-t_{\alpha / 2}<\frac{b_{k}-\beta_{k}}{\mathrm{SE}\left(b_{k}\right)}<t_{\alpha / 2}\right\}=1-\alpha\tag{9}$$

$$P\left\{b_{k}-t_{\alpha / 2} \operatorname{SE}\left(b_{k}\right)<\boldsymbol{\beta}_{k}<b_{k}+t_{\alpha / 2} \operatorname{SE}\left(b_{k}\right)\right\}=1-\alpha\tag{10}$$

In [6]:
from scipy. stats import t

alpha = 0.05   # 置信度
nu = max(0,n-K)   # 自由度
tval = t.ppf(1-alpha/2,nu)   # 逆函数值
SE_b = np.sqrt(np.diag(Varb)).reshape(K,1)
bint = np.hstack((b-tval*SE_b,b+tval*SE_b))
print('95% 置信区间：\n', bint)

95% 置信区间：
 [[-245.37220852   -4.00842522]
 [   1.25201092    1.33409777]]


#### t 检验

$$t_{k} \equiv \frac{b_{k}-\bar{\beta}_{k}}{\mathrm{SE}\left(b_{k}\right)} \equiv \frac{b_{k}-\bar{\beta}_{k}}{\sqrt{s^{2}\left(X^{\prime} X\right)_{k k}^{-1}}} \sim t(n-K)\tag{11}$$

<div align=center><img src="https://lei-picture.oss-cn-beijing.aliyuncs.com/img/20200423133127.png" width="450"></div>

In [7]:
t_stat = b/SE_b
t_p = 2*(1-t.cdf(abs(t_stat),n-K))

print('t检验为：\n', t_stat)
print('\n')
print('p值为：\n', t_p)

t检验为：
 [[-2.0304294 ]
 [61.91138223]]


p值为：
 [[0.0428902]
 [0.       ]]


#### 两类错误

根据样本信息对总体进行推断，有可能犯错误。特别地，在进行假设检验时，可能犯两类性质不同的错误。

**第Ⅰ类错误**：虽然原假设为真，但却根据观测数据做出了拒绝原假设的错误判断，即“弃真”。

**第Ⅱ类错误**：虽然原假设为假（替代假设为真），但却根据观测数据做出了接受原假设的错误判断，即“存伪”。

由于在进行假设检验时，通常知道第Ⅰ类错误的发生概率，而不知道第Ⅱ类错误的发生概率。因此，如果拒绝原假设，可以比较理直气壮，因为知道犯错误的概率就是显著性水平（比如5%）；另一方面，如果接受原假设，则比较没有把握，因为我们通常并不知道第Ⅱ类错误的发生概率（可能很高）。

## 使用 statsmodels 库实现

In [8]:
import statsmodels.api as sm

# sm.add_constant(data, prepend=False)
mod = sm.OLS(y, X)
res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.893
Model:                            OLS   Adj. R-squared:                  0.893
Method:                 Least Squares   F-statistic:                     3833.
Date:                Thu, 23 Apr 2020   Prob (F-statistic):          1.20e-224
Time:                        14:24:33   Log-Likelihood:                -2810.3
No. Observations:                 460   AIC:                             5625.
Df Residuals:                     458   BIC:                             5633.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -124.6903     61.411     -2.030      0.043    -245.372      -4.008
x1             1.2931      0.021     61.911      0.000       1.252       1.334
==============================================================================
Omnibus:                       61.627   Durbin-Watson:                   0.010
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               83.387
Skew:                           1.031   Prob(JB):                     7.81e-19
Kurtosis:                       2.692   Cond. No.                     3.55e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.55e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## matlab实现

```matlab
function [B,resid,siga2,bint,cov_matrix,t,t_p] = OLS_regress(Y,X)

%输入变量:
%Y - 被解释变量
%X - 解释变量

%输出变量:
% B - 待估计参数beta
% resid - 残差
% siga2 - 残差方差
% bint - 95%置信区间序列
% cov_matrix - 协方差矩阵

% 1.求OLS估计量B
B = inv(X'*X)*X'*Y;

% 2.计算协方差矩阵
resid = Y - X*B;  %残差
[n,K] = size(X);  
siga2 = sum(resid.^2)/(n-K);
cov_matrix = siga2*inv(X'*X);

% 3.t检验
t = B./sqrt(diag(cov_matrix));
t_p = 2*(1-tcdf(abs(t),n-K));

% 4.计算95%置信区间
alpha = 0.05;   %置信度
nu = max(0,n-K);  %自由度
tval = tinv(1-alpha/2,nu);
se = sqrt(diag(cov_matrix));
bint = [B-tval*se, B+tval*se];
```